In [1]:
# method of loop for greater volume of posts by David Little: 4-Step Reddit vs. NY Stock Exchange Sentiment Analysis

In [2]:
!pip install praw

In [3]:
import pandas as pd
import requests
import time
import praw

In [4]:
reddit = praw.Reddit(
    client_id="Jef0auDOdnMZ9vD6zlcWRg",
    client_secret="XqXCuNc590taBMgauwFmMBpDqvZp0A",
    user_agent="poppaearedditapilink",
    redirect_uri="http://localhost:8080",
)
print(reddit.auth.url(["identity"], "...", "permanent"))

https://www.reddit.com/api/v1/authorize?client_id=Jef0auDOdnMZ9vD6zlcWRg&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=identity&state=...


In [5]:
def submissions_pushshift_praw(subreddit, start=None, end=None, limit=10000, extra_query=""):
    """
    A simple function that returns a list of PRAW submission objects during a particular period from a defined sub.
    This function serves as a replacement for the now deprecated PRAW `submissions()` method.
    :param subreddit: A subreddit name to fetch submissions from.
    :param start: A Unix time integer. Posts fetched will be AFTER this time. (default: None)
    :param end: A Unix time integer. Posts fetched will be BEFORE this time. (default: None)
    :param limit: There needs to be a defined limit of results (default: 100), or Pushshift will return only 25.
    :param extra_query: A query string is optional. If an extra_query string is not supplied,
                        the function will just grab everything from the defined time period. (default: empty string)
    Submissions are yielded newest first.
    For more information on PRAW, see: https://github.com/praw-dev/praw
    For more information on Pushshift, see: https://github.com/pushshift/api
    """
    matching_praw_submissions = []
    
        # Default time values if none are defined (credit to u/bboe's PRAW `submissions()` for this section)
    utc_offset = 0
    now = int(time.time())
    start = max(int(start) + utc_offset if start else 0, 0)
    end = min(int(end) if end else now, now) + utc_offset

    # Format our search link properly.
    search_link = ('https://api.pushshift.io/reddit/submission/search/'
                   '?subreddit={}&after={}&before={}&sort_type=created_utc&sort=asc&limit={}&q={}')
    search_link = search_link.format(subreddit, start, end, limit, extra_query)

    # Get the data from Pushshift as JSON.
    retrieved_data = requests.get(search_link)
    returned_submissions = retrieved_data.json()['data']

    # Iterate over the returned submissions to convert them to PRAW submission objects.
    for submission in returned_submissions:
        # Take the ID, fetch the PRAW submission object, and append to our list
        praw_submission = reddit.submission(id=submission['id'])
        matching_praw_submissions.append(praw_submission)

    # Return all PRAW submissions that were obtained.
    return matching_praw_submissions


In [6]:
start_date = 1635724800 # 01/09/11 00:00:00
end_date = 1638316800  # 01/10/11 00:00:00

In [7]:
posts = pd.DataFrame(columns=['title', 'body', 'created'])  # Dataframe to store results

In [8]:
while start_date < end_date:  # Continue loop until end date is reached
    S = submissions_pushshift_praw(subreddit='stocks', start=start_date, end=end_date)  # Pull posts within date range
    for post in S:  # Looping through each post
        try:  # Try/except to catch any erroneous post pulls
            if post.selftext != '[removed]' and post.selftext != '[deleted]':  # Remove the deleted posts


                    posts = posts.append(
                        {'title': post.title,
                         'body': post.selftext,
                         'created': post.created}, ignore_index=True)  # Retrieve post data and append to dataframe
        except:
            next()  # Continue loop if error is found

    if len(S) < 100:  # To identify when the last pull is reached
        break

    start_date = posts['created'].max()  # Select the next earliest date to pull posts from
    print(start_date)  # An indicator of progress

1635778132.0


TypeError: next expected at least 1 argument, got 0

In [ ]:
posts

In [ ]:
posts['created'] = pd.to_datetime(posts['created'],unit='s')  # Converting Unix time to human readable

In [ ]:
posts

In [ ]:
df = posts

In [ ]:
df["text"] = df["title"] + df["body"]

In [ ]:
df = df.drop(["title", "body"], axis = 1)

In [ ]:
df

In [ ]:
!pip install TextBlob
from textblob import TextBlob

def preprocess(text):
    text = text.str.replace("(<br/>)", "")
    text = text.str.replace('(<a).*(>).*(</a>)', '')
    text = text.str.replace('(&amp)', '')
    text = text.str.replace('(&gt)', '')
    text = text.str.replace('(&lt)', '')
    text = text.str.replace('(\xa0)', ' ')  
    return text
df['text'] = preprocess(df['text'])

df['polarity'] = df['text'].map(lambda text: TextBlob(text).sentiment.polarity)
df['review_len'] = df['text'].astype(str).apply(len)
df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

In [ ]:
df

In [ ]:
df = df.drop(["review_len", "word_count", "text"], axis = 1)

In [ ]:
df

In [ ]:
df.to_csv(r'C:\Users\poppa\OneDrive - University of Bristol\3rd\data science\project\r_stocks_data\current\month_polarity\compile\nov3.csv', index = False)